# Lab 2 Widgets

In [1]:
# NO CODE
from __future__ import print_function
import datascience as ds
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import utils
plt.style.use('seaborn-muted')
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from matplotlib import patches
import csaps

In [2]:
individual_firm_costs = ds.Table.read_table('individual_firm_costs.csv')

In [3]:
total_cost = individual_firm_costs.column("Total Fixed Cost") + individual_firm_costs.column("Total Variable Cost")

In [4]:
# adding the array to the table
individual_firm_costs = individual_firm_costs.with_column("Total Cost", total_cost)

In [5]:
individual_firm_costs_no_first = individual_firm_costs.take(np.arange(1, individual_firm_costs.num_rows))
average_fixed_cost = individual_firm_costs_no_first.column("Total Fixed Cost") / individual_firm_costs_no_first.column("Output")
average_fixed_cost = np.append(ds.make_array(0), average_fixed_cost)

In [6]:
individual_firm_costs = individual_firm_costs.with_column("Average Fixed Cost", average_fixed_cost)

In [7]:
average_variable_cost = individual_firm_costs_no_first.column("Total Variable Cost")/individual_firm_costs_no_first.column("Output")
average_variable_cost = np.append(ds.make_array(0), average_variable_cost)

In [8]:
individual_firm_costs = individual_firm_costs.with_column("Average Variable Cost", average_variable_cost)

In [9]:
average_total_cost = individual_firm_costs_no_first.column("Total Cost")/individual_firm_costs_no_first.column("Output")
average_total_cost = np.append(ds.make_array(0), average_total_cost)

In [10]:
individual_firm_costs = individual_firm_costs.with_column("Average Total Cost", average_total_cost)

In [11]:
marginal_cost = np.diff(total_cost)
marginal_cost = np.append(ds.make_array(0), marginal_cost)

In [12]:
individual_firm_costs = individual_firm_costs.with_column("Marginal Cost", marginal_cost)

## Lab 2 Widget 1

In [13]:
interact(lambda price: utils.firm_behaviour(price, individual_firm_costs), price=widgets.IntSlider(min=20,max=60,step=1,value=25));

interactive(children=(IntSlider(value=25, description='price', max=60, min=20), Output()), _dom_classes=('widg…

In [14]:
# NO CODE
ESG_table = ds.Table.read_table('ESGPorfolios_forcsv.csv').select(
    "Group", "Group_num", "UNIT NAME", "Capacity_MW", "Total_Var_Cost_USDperMWH").sort(
    "Total_Var_Cost_USDperMWH", descending = False).relabel(4, "Average Variable Cost")

In [15]:
selection = 'Big Coal'
Group = ESG_table.where("Group", selection)

## Lab 2 Widget 2

In [16]:
interact(utils.group_plot, price=widgets.IntSlider(min=20,max=80,step=1,value=37));

interactive(children=(IntSlider(value=37, description='price', max=80, min=20), Output()), _dom_classes=('widg…

## Lab 2 Widget 3

In [17]:
interact(utils.ESG_plot, price=widgets.IntSlider(min=0,max=90,step=1,value=37));

interactive(children=(IntSlider(value=37, description='price', max=90), Output()), _dom_classes=('widget-inter…

## Lab 3 Widget 1

In [19]:
from datascience import *

import sympy
solve = lambda x,y: sympy.solve(x-y)[0] if len(sympy.solve(x-y))==1 else "Not Single Solution"
import matplotlib.pyplot as plt
plt.style.use('seaborn-muted')
%matplotlib inline

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
from ipywidgets import interact


In [29]:
airlines = Table.read_table('airlines.csv')

In [21]:
airlines = airlines.drop("Price")
airlinesFiltered = airlines.relabeled(["Average Fare", "Average Weekly Passengers"], ["Price", "Quantity"])

The following cell of code is provided.

In [22]:
delta = airlines.where("Market Leading Airline", "DL")
delta = delta.select(["Average Fare", "Average Weekly Passengers"])
delta = delta.relabeled(["Average Weekly Passengers", "Average Fare"], ["Quantity", "Price"])
delta = delta.where(delta.column("Quantity") < 2000)

In [23]:
Q = sympy.Symbol("Q")

In [24]:
deltademand = -0.035 * Q + 189.1

In [25]:
deltasupply = 0.03 * Q + 14

In [26]:
Q_star_delta = solve(deltademand, deltasupply)

In [27]:
supply = 0.0225 * Q + 100

In [28]:
def plot_equation(equation, price_start, price_end, label=None):
    plot_prices = [price_start, price_end]
    plot_quantities = [equation.subs(list(equation.free_symbols)[0], c) for c in plot_prices]
    plt.plot(plot_prices, plot_quantities, label=label)
    
def plot_intercept(eq1, eq2):
    ex = sympy.solve(eq1-eq2)[0]
    why = eq1.subs(list(eq1.free_symbols)[0], ex)
    plt.scatter([ex], [why])
    return (ex, why)
    
def shift_demand():
    equation = deltademand
    def shift_helper(shift):
        plot_equation(equation, 0, 10000, "Original")
        plot_equation(deltasupply, 0, 10000)
        old = plot_intercept(equation, deltasupply)
        print('Original Intercept:', old)
        
        if shift != 0:
            plot_equation(equation + shift, -10, 10000, 'Shifted')
            new = plot_intercept(equation + shift, supply)
            print('New intercept:', new)
            print('Change in Price:', round(float(new[1]-old[1]), 2))
        else:
            print('Nothing shifted yet, use the slider to move the line!')
        plt.xlim(0,5000)
        plt.ylim(0,200)
        plt.legend()
        plt.ylabel("Price")
        plt.xlabel("Quantity")
        plt.title("Shift in Customer Preference Impact on Delta Ticket Sales")
    interact(shift_helper, shift=(-50, 40, 2))

shift_demand()

interactive(children=(IntSlider(value=-6, description='shift', max=40, min=-50, step=2), Output()), _dom_class…